1. Faire les tokens
Après cette étape, il faut faire quelques nettoyages :
2. Enlever certains mots qui peuvent être de la même famille ou exprimer le même sens (participes passé avec accord, pluriels…) Ex : multivarié - multivariée. Technique Stemming ou lemmatisation, sinon avec la distance de Distance de Levenshtein.
3. Concatener la liste de tokens avec un dictionnaire de la langue française et ensuite soustraire les mots en commun pour obtenir seulement les mots techniques (langages de programmation, skills en anglais, etc.).
4. Enlever les doublons
5. Enlever les noms des villes et régions de France avec la même méthode d’un dictionnaire déjà construit.

In [20]:
import pandas as pd
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import numpy as np

In [21]:
data = pd.read_json('BDD_14_10_2019.json',lines=True,encoding='utf-8')
df = data.loc[data['Salaire']!='None',:]
df = df.loc[df['Descriptif_du_poste']!="None",:]

In [22]:
df.head()

,Date_de_publication,Descriptif_du_poste,Entreprise,Salaire,Scrapped_job,Scrapped_location,Titre,Type_de_contrat,Ville,_id
0,il y a 4 jours,#JobAlert #Stats #DataScientist #Nantes #Mérie...,SILLIKER - MERIEUX NUTRISCIENCES FRANCE,40 000 € par an,data scientist,Nantes,Data Scientist H/F - BIOFORTIS - CDI - Saint-H...,None,Saint-Herblain (44),{'$oid': '5d9b7361130e0588f823d767'}
4,il y a 4 jours,"KANDY, basée à Treillières, près de Nantes (44...",KANDY,30 000 € - 40 000 € par an,data scientist,Nantes,Data Scientist H/F,None,Treillières (44),{'$oid': '5d9b7363130e0588f823d76b'}
5,il y a 4 jours,"A compétences égales, le poste est ouvert aux ...",EPSILON-ALCEN,30 000 € - 50 000 € par an,data scientist,Nantes,Ingénieur Data Scientist H/F,None,Nantes (44),{'$oid': '5d9b7364130e0588f823d76c'}
11,il y a 4 jours,#JobAlert #Stats #DataScientist #Nantes #Mérie...,SILLIKER - MERIEUX NUTRISCIENCES FRANCE,40 000 € par an,data scientist,Nantes,Data Scientist H/F - BIOFORTIS - CDI - Saint-H...,CDI,Saint-Herblain (44),{'$oid': '5d9b7396ac24ca882fb2d285'}
13,il y a 4 jours,"A compétences égales, le poste est ouvert aux ...",EPSILON-ALCEN,30 000 € - 50 000 € par an,data scientist,Nantes,Ingénieur Data Scientist H/F,CDI,Nantes (44),{'$oid': '5d9b739aac24ca882fb2d287'}


In [23]:
df.shape

(2249, 10)

## Tokens

In [32]:
from gensim.utils import tokenize
df["Tokens_description"] = df.apply(lambda x: list(tokenize(x['Descriptif_du_poste'],lowercase=True)), axis=1)

In [31]:
descriptions = df["Tokens_description"][0]
descriptions

['jobalert',
 'stats',
 'datascientist',
 'nantes',
 'mérieuxnutrisciences',
 'biofortis',
 'basée',
 'à',
 'saint',
 'herblain',
 'près',
 'de',
 'nantes',
 'est',
 'la',
 'filiale',
 'du',
 'groupe',
 'merieux',
 'nutrisciences',
 'depuis',
 'présent',
 'dans',
 'pays',
 'et',
 'à',
 'travers',
 'plus',
 'de',
 'laboratoires',
 'merieux',
 'nutrisciences',
 'fait',
 'avancer',
 'la',
 'science',
 'en',
 'proposant',
 'des',
 'solutions',
 'pour',
 'la',
 'qualité',
 'et',
 'sécurité',
 'alimentaire',
 'notre',
 'vision',
 'est',
 'en',
 'effet',
 'd',
 'améliorer',
 'la',
 'santé',
 'grâce',
 'à',
 'des',
 'aliments',
 'plus',
 'sûrs',
 'et',
 'une',
 'nutrition',
 'plus',
 'saine',
 'cette',
 'vision',
 'se',
 'concrétise',
 'chaque',
 'jour',
 'grâce',
 'à',
 'nos',
 'collaborateurs',
 'animés',
 'par',
 'nos',
 'valeurs',
 'l',
 'excellence',
 'l',
 'intégrité',
 'le',
 'sens',
 'de',
 'l',
 'initiative',
 'et',
 'le',
 'sens',
 'des',
 'responsabilités',
 'basée',
 'à',
 'regroup

## Enlever certains mots qui peuvent être de la même famille

In [36]:
# Stemming
from nltk.stem import PorterStemmer
ps =PorterStemmer()

In [38]:
for w in descriptions:
    rootWord=ps.stem(w)
    #print(w)
    #print(rootWord)
    print(w, " : ", ps.stem(w))

jobalert  :  jobalert
stats  :  stat
datascientist  :  datascientist
nantes  :  nant
mérieuxnutrisciences  :  mérieuxnutrisci
biofortis  :  bioforti
basée  :  basé
à  :  à
saint  :  saint
herblain  :  herblain
près  :  prè
de  :  de
nantes  :  nant
est  :  est
la  :  la
filiale  :  filial
du  :  du
groupe  :  group
merieux  :  merieux
nutrisciences  :  nutrisci
depuis  :  depui
présent  :  présent
dans  :  dan
pays  :  pay
et  :  et
à  :  à
travers  :  traver
plus  :  plu
de  :  de
laboratoires  :  laboratoir
merieux  :  merieux
nutrisciences  :  nutrisci
fait  :  fait
avancer  :  avanc
la  :  la
science  :  scienc
en  :  en
proposant  :  propos
des  :  de
solutions  :  solut
pour  :  pour
la  :  la
qualité  :  qualité
et  :  et
sécurité  :  sécurité
alimentaire  :  alimentair
notre  :  notr
vision  :  vision
est  :  est
en  :  en
effet  :  effet
d  :  d
améliorer  :  amélior
la  :  la
santé  :  santé
grâce  :  grâce
à  :  à
des  :  de
aliments  :  aliment
plus  :  plu
sûrs  :  sûr
et 

permis  :  permi
b  :  b
souhaité  :  souhaité
langue  :  langu
anglais  :  anglai
souhaité  :  souhaité


## Lemmatisation

In [39]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer()

In [42]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ludovicbenard/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [44]:
for w in descriptions:
    #rootWord=ps.stem(w)
    #print(w)
    #print(rootWord)
    #print(w, " : ", ps.stem(w))
    print(w, " : ", lemmatizer.lemmatize(w)) 

jobalert  :  jobalert
stats  :  stats
datascientist  :  datascientist
nantes  :  nantes
mérieuxnutrisciences  :  mérieuxnutrisciences
biofortis  :  biofortis
basée  :  basée
à  :  à
saint  :  saint
herblain  :  herblain
près  :  près
de  :  de
nantes  :  nantes
est  :  est
la  :  la
filiale  :  filiale
du  :  du
groupe  :  groupe
merieux  :  merieux
nutrisciences  :  nutrisciences
depuis  :  depuis
présent  :  présent
dans  :  dans
pays  :  pay
et  :  et
à  :  à
travers  :  travers
plus  :  plus
de  :  de
laboratoires  :  laboratoires
merieux  :  merieux
nutrisciences  :  nutrisciences
fait  :  fait
avancer  :  avancer
la  :  la
science  :  science
en  :  en
proposant  :  proposant
des  :  de
solutions  :  solution
pour  :  pour
la  :  la
qualité  :  qualité
et  :  et
sécurité  :  sécurité
alimentaire  :  alimentaire
notre  :  notre
vision  :  vision
est  :  est
en  :  en
effet  :  effet
d  :  d
améliorer  :  améliorer
la  :  la
santé  :  santé
grâce  :  grâce
à  :  à
des  :  de
alimen

## Distance de Levenshtein

In [60]:
import Levenshtein as lev
d = lev.distance("tirao", "tiraos")
d

1

In [61]:
# Pour chercher tous les mots qui ont une lettre de plus ou de moins (à une lettre d'écart)
# Si les mots ont un écart de un, on peut effacer le mot

t = [[w,wf] for w in descriptions for wf in descriptions if abs(len(wf)-len(w))==1]
t

[['jobalert', 'biofortis'],
 ['jobalert', 'filiale'],
 ['jobalert', 'merieux'],
 ['jobalert', 'présent'],
 ['jobalert', 'travers'],
 ['jobalert', 'merieux'],
 ['jobalert', 'avancer'],
 ['jobalert', 'science'],
 ['jobalert', 'proposant'],
 ['jobalert', 'solutions'],
 ['jobalert', 'qualité'],
 ['jobalert', 'améliorer'],
 ['jobalert', 'nutrition'],
 ['jobalert', 'valeurs'],
 ['jobalert', 'intégrité'],
 ['jobalert', 'fonctions'],
 ['jobalert', 'support'],
 ['jobalert', 'expertise'],
 ['jobalert', 'technique'],
 ['jobalert', 'merieux'],
 ['jobalert', 'fournit'],
 ['jobalert', 'seulement'],
 ['jobalert', 'qualité'],
 ['jobalert', 'surtout'],
 ['jobalert', 'solutions'],
 ['jobalert', 'pratiques'],
 ['jobalert', 'biofortis'],
 ['jobalert', 'nutrition'],
 ['jobalert', 'clients'],
 ['jobalert', 'offrant'],
 ['jobalert', 'originaux'],
 ['jobalert', 'innovants'],
 ['jobalert', 'preuves'],
 ['jobalert', 'cliniques'],
 ['jobalert', 'cliniques'],
 ['jobalert', 'cliniques'],
 ['jobalert', 'biofortis']